## Create precinct level geojson files for each race in the June 7, 2022 primary election in Los Angeles County

This notebook loads a geodatabase and election results from the LA County RRCC and outputs a geojson file of all of the precincts in that race.

The geodatabase was mailed to me on a CD, but the results are downloaded directly from the RRCC website.

In [9]:
# load the libraries we need
import pandas as pd
import geopandas as gp
import fiona
from glob import glob
from zipfile import ZipFile
import requests
from io import BytesIO

**Load the geodatabase**

In [23]:
# Load the database file
precincts = (
    gp.read_file("DavidYanofsky.gdb", layer="June2022Primary_ElectionPrecincts")

    # reproject the data into lat lons
    .to_crs("epsg:4326")

)

precincts.head()

,Code,Label,ElectionName,CheckInCenterCode,CheckInCenterName,CheckInCenterAddress,CheckInCenterLocation,CheckInCenterCity,CheckInCenterZipCode,CheckInCenterComment,...,StopNumberValidation,TroubleShootNumberValidation,CoordinatorNumberValidation,PollValidation,DistanceFromPoll,DistanceFromCheckInCenter,CheckInCenterPollDistance,shape_Length,shape_Area,geometry
0,0090007C,0090007C,STATEWIDE DIRECT PRIMARY ELECTION,None,None,None,None,None,None,None,...,-1,-1,-1,-1,None,NaN,None,15545.857292,8.250596e+06,"MULTIPOLYGON (((-118.37949 34.46781, -118.3797..."
1,6300006E,6300006E,STATEWIDE DIRECT PRIMARY ELECTION,None,None,None,None,None,None,None,...,-1,-1,-1,-1,None,NaN,None,47737.378050,7.358780e+07,"MULTIPOLYGON (((-118.46540 34.46649, -118.4665..."
2,9006376A,9006376A,STATEWIDE DIRECT PRIMARY ELECTION,None,None,None,None,None,None,None,...,0,0,0,0,None,NaN,None,11024.677072,6.093174e+06,"MULTIPOLYGON (((-118.30115 33.79201, -118.3011..."
3,9003803A,9003803A,STATEWIDE DIRECT PRIMARY ELECTION,None,None,None,None,None,None,None,...,-1,-1,-1,-1,None,NaN,None,2725.043627,2.773679e+05,"MULTIPOLYGON (((-118.29992 33.79781, -118.2991..."
4,9006370A,9006370A,STATEWIDE DIRECT PRIMARY ELECTION,None,None,None,None,None,None,None,...,0,0,0,0,None,NaN,None,14582.711966,9.008198e+06,"MULTIPOLYGON (((-118.29898 33.80411, -118.2990..."


**Download the precinct level results**

In [24]:
# Download and unzip the precinct level results.

# via https://www.lavote.gov/home/voting-elections/current-elections/election-results/past-election-results
election_results_url = "https://content.lavote.gov/docs/rrcc/svc/4269_final_svc_excel.zip?v=2"

# download the file
r = requests.get(election_results_url)

# create a zipfile object
z = ZipFile(BytesIO(r.content))

# unzip the file into a new directory
z.extractall("results/")

**Create the geojson files**

In [25]:
# make sure the directory we need is there
!mkdir geojson-by-race

mkdir: geojson-by-race: File exists


In [26]:
# loop through every results file
for fn in glob("results/*.xls"):
    # get the name of the file without the extension or directory
    # this is the name of the race
    race_slug = fn.split("/")[-1].split(".xls")[0]

    # load the results as a dataframe
    df = pd.read_excel(fn, skiprows=2)

    # make a list of all of the precincts that voted in the election
    district_precincts = df.PRECINCT.unique()

    # filter the precincts to only those that are in the election
    gdf = precincts.query("Label.isin(@district_precincts)")

    # save the filtered geodata to a geojson file
    gdf.to_file(f"geojson-by-race/{race_slug}.geojson")

Joining the results data with the precinct data is more complicated, here's how you might do that.

In [27]:
# make sure the destination directory exists
!mkdir geojson-by-race-with-results

# define the variables that are informational rather than results
info_variables = ["BALLOTS CAST", "REGISTRATION", "VOTE BY MAIL ONLY"]

# create a function to extract slices of data from the results file
def isolate_data_slice(melted_df, type_key, column_name, is_info):
    """
    There's are multiple types of results information for every precinct
    This takes the melted results and returns it in a way that it can be joined
    with the geodata so that the geojsons are easy to work with
    """
    out_df = (
        melted_df.query(f"TYPE == '{type_key}'")
            .drop(columns=["TYPE"])
            .pivot(columns="variable", values="value")
    )

    if is_info:
        # if this is an informational variable only keep the info data
        out_df = out_df.filter(info_variables)
    else:
        # if this is results data, only keep the results data
        out_df = out_df.drop(columns=info_variables)

    return (
        out_df
            .assign(**{column_name:lambda x: x.apply(lambda row: row.to_dict(),axis=1)})
            .filter([column_name])
    )


# loop through every results file
for fn in glob("results/*.xls"):
    # get the name of the file without the extension or directory
    # this is the name of the race
    race_slug = fn.split("/")[-1].split(".xls")[0]

    # load the results as a dataframe
    df = pd.read_excel(fn, skiprows=2)

    # remove the LOCATION, SERIAL, BALLOT GROUP, and any Unnamed column
    unnamed_columns = list(filter(lambda x: "Unnamed" in x, df.columns))
    columns_to_drop = ["LOCATION", "SERIAL", "BALLOT GROUP"] + unnamed_columns
    df = df.drop(columns=columns_to_drop)

    # make a list of all of the precincts that voted in the election
    district_precincts = df.PRECINCT.unique()

    # melt the dataframe so that each precinct-type combination has its own row
    melted = df.melt(id_vars=["PRECINCT", "TYPE"]).set_index("PRECINCT")

    
    gdf = (
        # filter the precincts to only those that are in the election
        precincts.query("Label.isin(@district_precincts)")

            # set the index to the precinct number
            .set_index("Label")

            # add total, vote by mail, and in-person to each precinct 
            # for BALLOTS CAST, REGISTRATION, and VOTE BY MAIL ONLY
            .join(isolate_data_slice(melted, "TOTAL", "info_total", True))
            .join(isolate_data_slice(melted, "VBM PORTION", "info_vbm", True))
            .join(isolate_data_slice(melted, "POLLING PLACE", "info_polling_place", True))
            
            # add total, vote by mail, and in-person results data to each precinct
            .join(isolate_data_slice(melted, "TOTAL", "results_total", False))
            .join(isolate_data_slice(melted, "VBM PORTION", "results_vbm", False))
            .join(isolate_data_slice(melted, "POLLING PLACE", "results_polling_place", False))
    )

    # save the filtered geodata to a geojson file
    gdf.to_file(f"geojson-by-race-with-results/{race_slug}.geojson")
        

mkdir: geojson-by-race-with-results: File exists
